In [3]:
## Source
## https://deeplizard.com/learn/video/QK_PP_2KgGE

In [4]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [5]:
env = gym.make("FrozenLake-v0")

In [6]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [7]:
print(action_space_size)
print(state_space_size)
print(q_table)

4
16
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [8]:
num_episodes = 1000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [9]:
def q_learning(q_table):
    for episode in range(3):
        state = env.reset()
        done = False
        print("*****EPISODE ", episode+1, "*****\n\n\n\n")
        time.sleep(1)
        for step in range(max_steps_per_episode):        
            clear_output(wait=True)
            env.render()
            time.sleep(0.3)

            action = np.argmax(q_table[state,:])        
            new_state, reward, done, info = env.step(action)

            if done:
                clear_output(wait=True)
                env.render()
                if reward == 1:
                    print("****You reached the goal!****")
                    time.sleep(2)
                else:
                    print("****You fell through a hole!****")
                    time.sleep(1)
                clear_output(wait=True)
                break

            state = new_state

    env.close()

In [10]:
def basic(q_table,exploration_rate,num_episodes,learning_rate,discount_rate):
    
    rewards_all_episodes = []

    # Q-Learning algorithm
    for episode in range(num_episodes):
        state = env.reset()

        done = False
        rewards_current_episode = 0

        for step in range(max_steps_per_episode): 

            # Exploration-exploitation trade-off
            exploration_rate_threshold = random.uniform(0, 1)
            if exploration_rate_threshold > exploration_rate:
                action = np.argmax(q_table[state,:]) # Exploitation
            else:
                action = env.action_space.sample() # Exploration

            new_state, reward, done, info = env.step(action)

            # Update Q-table for Q(s,a)
            q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
                learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))

            state = new_state
            rewards_current_episode += reward


            if done == True: 
                break


        # Exploration rate decay
        exploration_rate = min_exploration_rate + \
            (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)


        rewards_all_episodes.append(rewards_current_episode)


    # Calculate and print the average reward per thousand episodes
    rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
    count = 1000

    print("********Average reward per thousand episodes********\n")
    for r in rewards_per_thousand_episodes:
        print(count, ": ", str(sum(r/1000)))
        count += 1000
    
    return q_table

In [11]:
def sarsa(q_table,exploration_rate,num_episodes,learning_rate,discount_rate):
    
    rewards_all_episodes = []

    # Q-Learning algorithm
    for episode in range(num_episodes):
        state = env.reset()

        done = False
        rewards_current_episode = 0

        for step in range(max_steps_per_episode): 

            # Exploration-exploitation trade-off
            exploration_rate_threshold = random.uniform(0, 1)
            if exploration_rate_threshold > exploration_rate:
                action = np.argmax(q_table[state,:]) # Exploitation
            else:
                action = env.action_space.sample() # Exploration

            new_state, reward, done, info = env.step(action)


            # Sarsa: An on-policy TD control algorithm.
            q_table[state, action] = q_table[state, action]  + \
                learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]) - (q_table[state, action]))

            state = new_state
            rewards_current_episode += reward


            if done == True: 
                break


        # Exploration rate decay
        exploration_rate = min_exploration_rate + \
            (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)


        rewards_all_episodes.append(rewards_current_episode)


    # Calculate and print the average reward per thousand episodes
    rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
    count = 1000

    print("********Average reward per thousand episodes********\n")
    for r in rewards_per_thousand_episodes:
        print(count, ": ", str(sum(r/1000)))
        count += 1000
    
    return q_table

In [12]:
def refactoryperiod(q_table,exploration_rate,num_episodes,learning_rate,discount_rate):
    
    rewards_all_episodes = []

    # Q-Learning algorithm
    for episode in range(num_episodes):
        state = env.reset()

        done = False
        rewards_current_episode = 0
        
        refactory_period = 5

        for step in range(max_steps_per_episode): 

            # Exploration-exploitation trade-off
            exploration_rate_threshold = random.uniform(0, 1)
            if exploration_rate_threshold > exploration_rate:
                action = np.argmax(q_table[state,:]) # Exploitation
            else:
                action = env.action_space.sample() # Exploration

            new_state, reward, done, info = env.step(action)


            if(refactory_period == 0):
                # Sarsa: An on-policy TD control algorithm.
                q_table[state, action] = q_table[state, action]  + \
                    learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]) - (q_table[state, action]))
                
                refactory_period = 5
                
            refactory_period = refactory_period - 1

            state = new_state
            rewards_current_episode += reward


            if done == True: 
                break


        # Exploration rate decay
        exploration_rate = min_exploration_rate + \
            (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)


        rewards_all_episodes.append(rewards_current_episode)


    # Calculate and print the average reward per thousand episodes
    rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
    count = 1000

    print("********Average reward per thousand episodes********\n")
    for r in rewards_per_thousand_episodes:
        print(count, ": ", str(sum(r/1000)))
        count += 1000
    
    return q_table

In [13]:
new_qtable = basic(q_table,exploration_rate,num_episodes,learning_rate,discount_rate)
# Print updated Q-table
print("\n\n******** Basic Q-table********\n")
print(new_qtable)

********Average reward per thousand episodes********

1000 :  0.03900000000000003


******** Basic Q-table********

[[0.26559475 0.23274041 0.24564058 0.24158167]
 [0.12776628 0.11074493 0.13451671 0.19057206]
 [0.16974807 0.0758261  0.06986535 0.08831105]
 [0.02216495 0.02746777 0.02271978 0.07125702]
 [0.28365454 0.13692225 0.20646591 0.19683609]
 [0.         0.         0.         0.        ]
 [0.04187798 0.00792743 0.18040892 0.00625731]
 [0.         0.         0.         0.        ]
 [0.11663474 0.21402043 0.19327489 0.32659224]
 [0.11112157 0.40304968 0.21934775 0.13184289]
 [0.36172744 0.13473909 0.11837375 0.10243724]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.19867333 0.26404278 0.52370948 0.21262982]
 [0.25497666 0.38137619 0.34084856 0.73639774]
 [0.         0.         0.         0.        ]]


In [14]:
q_learning(new_qtable)

  (Up)
SFFF
FHFH
FFFH
HFFG
****You reached the goal!****


In [15]:
new_qtable = sarsa(q_table,exploration_rate,num_episodes,learning_rate,discount_rate)
# Print updated Q-table
print("\n\n******** Sarsa Q-table********\n")
print(new_qtable)

********Average reward per thousand episodes********

1000 :  0.04200000000000003


******** Sarsa Q-table********

[[0.42856725 0.38610366 0.38048682 0.38076119]
 [0.28767148 0.19595752 0.22958006 0.36009383]
 [0.29361006 0.25352963 0.26067328 0.28030392]
 [0.11544219 0.13592306 0.16877386 0.25596025]
 [0.44065154 0.30514626 0.38550196 0.26240043]
 [0.         0.         0.         0.        ]
 [0.18135853 0.16611273 0.24461295 0.09571862]
 [0.         0.         0.         0.        ]
 [0.26443513 0.32649613 0.24607827 0.46230674]
 [0.31721167 0.52497163 0.2877823  0.2489617 ]
 [0.54851515 0.38440643 0.36083512 0.25527655]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.38624921 0.40801744 0.63507975 0.35843081]
 [0.54025413 0.73346571 0.64451419 0.74436241]
 [0.         0.         0.         0.        ]]


In [16]:
q_learning(new_qtable)

  (Up)
SFFF
FHFH
FFFH
HFFG
****You reached the goal!****


In [17]:
new_qtable = refactoryperiod(q_table,exploration_rate,num_episodes,learning_rate,discount_rate)
# Print updated Q-table
print("\n\n******** Sarsa with Refactory Period Q-table********\n")
print(new_qtable)

********Average reward per thousand episodes********

1000 :  0.05500000000000004


******** Sarsa with Refactory Period Q-table********

[[0.44574817 0.41889471 0.43751341 0.42512771]
 [0.29170684 0.26719723 0.28346144 0.37117249]
 [0.3078411  0.30833751 0.28989709 0.32621517]
 [0.19489307 0.140767   0.1572782  0.30888375]
 [0.46448531 0.32032984 0.33051207 0.31907576]
 [0.         0.         0.         0.        ]
 [0.18308925 0.21149345 0.21460231 0.08137745]
 [0.         0.         0.         0.        ]
 [0.37480544 0.32424758 0.33242296 0.5117188 ]
 [0.30798565 0.53802043 0.41580786 0.33464519]
 [0.54329851 0.47000757 0.37275719 0.28171276]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.38408118 0.41366457 0.62127065 0.36719586]
 [0.5859829  0.74232636 0.72671425 0.72818954]
 [0.         0.         0.         0.        ]]


In [18]:
q_learning(new_qtable)

  (Right)
SFFF
FHFH
FFFH
HFFG
****You fell through a hole!****
